In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import requests
import os
from PIL import Image
import re

In [27]:
url = "https://readcomiconline.li/Comic/Injustice-Gods-Among-Us-Year-Two"
parsed_url = url.split("/")
comic_series = parsed_url[-1]

In [28]:


options = webdriver.FirefoxOptions()
options = options.add_argument("-headless")
driver = webdriver.Firefox(options=options)
driver.get(url)
html_content = driver.page_source



In [29]:
table_element = driver.find_element(By.CLASS_NAME, "listing")
anchor_elements = table_element.find_elements(By.TAG_NAME, "a")
# issues_urls = [elem.get_attribute("href") for elem in anchor_elements]
# issues = {elem.text: elem.get_attribute("href") for elem in anchor_elements}
issues = [{elem.text: elem.get_attribute("href")} for elem in anchor_elements]
issues.reverse()


driver.quit()


In [30]:
comic_series_folder = f"{comic_series}"
try:
    os.makedirs(comic_series_folder)
    print(f"Directory '{comic_series_folder}' created successfully.")
    comic_series_path = os.path.join(comic_series_folder)
except OSError as e:
    print(f"Error creating directory '{comic_series_folder}': {e}")
    # Optionally, handle the error or raise it to the caller.


Directory 'Injustice-Gods-Among-Us-Year-Two' created successfully.


In [32]:
for issue in issues[9:]:

    for title, url in issue.items():
        # if title[-2] != "#" and int(title[-2:]) > 7:
        if True:
        # if int(title[-2:]) > 22:
            # pass
        # else:
            try:
                pattern = r'[<>:"\/|?*]'
                # Use re.sub() to replace all occurrences of the special characters with an empty string
                title = re.sub(pattern, '', title)

                issue_folder = os.path.join(comic_series_path, title)
                images_folder = os.path.join(issue_folder, "images")
                os.makedirs(images_folder)
                print(f"Directories '{issue_folder}' and '{images_folder}' created successfully.")
            except OSError as e:
                print(f"Error creating directories: {e}")
                # Optionally, handle the error or raise it to the caller.

            print("Trying", title)

            options = webdriver.FirefoxOptions()
            options = options.add_argument("-headless")
            driver = webdriver.Firefox(options=options)
            driver.get(url)
            select_element_page = driver.find_element(By.ID, "selectReadType")
            select = Select(select_element_page)
            select.select_by_value("1")

            div_image = driver.find_element(By.ID, "divImage")
            img_elements = div_image.find_elements(By.TAG_NAME, "img")

            img_srcs = [elem.get_attribute("src") for elem in img_elements]

            driver.quit()

            image_files = []
            i = 1
            for img_url in img_srcs:
                response = requests.get(img_url)
                if response.status_code == 200:
                    image_name = f"{title}_image_{i}.jpg"
                    with open(os.path.join(images_folder, image_name), "wb") as image:
                        image_download = response.content
                        image.write(image_download)
                    pil_image = Image.open(os.path.join(images_folder, image_name))
                    image_files.append(pil_image)
                i += 1
            image_files[0].save(f"{comic_series_folder}/{title}.pdf", save_all=True, append_images=image_files[1:])
            print("Successful", title)


Error creating directories: [WinError 183] Cannot create a file when that file already exists: 'Injustice-Gods-Among-Us-Year-Two\\Injustice Gods Among Us Year Two Issue #9\\images'
Trying Injustice Gods Among Us Year Two Issue #9
Successful Injustice Gods Among Us Year Two Issue #9
Directories 'Injustice-Gods-Among-Us-Year-Two\Injustice Gods Among Us Year Two Issue #10' and 'Injustice-Gods-Among-Us-Year-Two\Injustice Gods Among Us Year Two Issue #10\images' created successfully.
Trying Injustice Gods Among Us Year Two Issue #10
Successful Injustice Gods Among Us Year Two Issue #10
Directories 'Injustice-Gods-Among-Us-Year-Two\Injustice Gods Among Us Year Two Issue #11' and 'Injustice-Gods-Among-Us-Year-Two\Injustice Gods Among Us Year Two Issue #11\images' created successfully.
Trying Injustice Gods Among Us Year Two Issue #11
Successful Injustice Gods Among Us Year Two Issue #11
Directories 'Injustice-Gods-Among-Us-Year-Two\Injustice Gods Among Us Year Two Issue #12' and 'Injustice-Go